In [4]:
# Importing libraries
from youtube_transcript_api import YouTubeTranscriptApi
from pymongo import MongoClient
import requests
from clearml import Task, PipelineController
# from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.http import models
from sentence_transformers import SentenceTransformer
import re

In [5]:
# Task.init(project_name="Featurization Pipeline", task_name="Featurization Pipeline")

In [6]:
from pymongo import MongoClient

# Connect to the MongoDB server
client = MongoClient("mongodb://localhost:27017/")  # Replace with your connection string

# Access the database and collection
db = client["media_data_final1"]
collection = db["raw_data_final"]

# Retrieve all records
records = collection.find()
count = collection.count_documents({})
print(count)
# Iterate through and print each record
# for record in records:
#     print(record)


2583


In [7]:
# Connecting to MongoDB

# MongoDB Configuration
MONGO_URI = "mongodb://localhost:27017/"
DB_NAME = "media_data_final1"
EMBEDDINGS_COLLECTION_NAME = "embeddings"
RAW_DATA_COLLECTION_NAME = "raw_data_final"

# Connect to MongoDB
mongo_client = MongoClient(MONGO_URI)
db = mongo_client[DB_NAME]
collection = db[RAW_DATA_COLLECTION_NAME]

In [8]:
# Connecting to Qdrant

qdrant_client = QdrantClient(":memory:")
collection_name = "text_collection"
qdrant_client.recreate_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(size=384, distance="Cosine")
)

C:\Users\harsh\AppData\Local\Temp\ipykernel_65704\4197240320.py:5: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant_client.recreate_collection(


True

In [9]:
def create_embeddings_and_persist(data, source, url):
    mongo_client = MongoClient(MONGO_URI)
    db = mongo_client[DB_NAME]
    collection = db[EMBEDDINGS_COLLECTION_NAME]
    model = SentenceTransformer('paraphrase-MiniLM-L6-v2') 
    embedding = model.encode(data)
    
    document = {
        "source": source,
        "url": url,
        "embedding": embedding.tolist(),
        "data": data
    }
    collection.insert_one(document)
    print("Embeddings stored into MongoDB")

In [10]:
def chunk_youtube_data(data):
    sentences = [i['text'] for i in data]
    chunks = " ".join(sentences)
    return chunks

In [11]:
def create_data_embeddings():
    client = MongoClient("mongodb://localhost:27017/") 

    # Access the database and collection
    db = client["media_data_final1"]
    collection = db["raw_data_final"]

    # Retrieve all records
    records = collection.find()
    count = 0
    for record in records:
        count+=1
        source = record.get("source")
        if source == "web":
            continue
            data = record.get("content")
        data = record.get("data")
        if (source == "youtube"):
            data = chunk_youtube_data(record.get("data"))
        create_embeddings_and_persist(data, source, record.get("url"))
        

In [12]:
create_data_embeddings()

Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddings stored into MongoDB
Embeddin

In [13]:
def clean_text(text):
    text = re.sub(r'<.*?>', '', text)  # Remove HTML tags
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Remove special characters
    return text.strip().lower()

def featurize_web_data():
    mongo_client = MongoClient(MONGO_URI)
    db = mongo_client[DB_NAME]
    embedding_collection = db[EMBEDDINGS_COLLECTION_NAME]
    model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
    web_data = collection.find({"source": "web"})
    for doc in web_data:
        sentences = doc['content'].split('\n')
        sentences = [clean_text(sentence) for sentence in sentences]
        paragraph = " ".join(sentences)
        paragraph = re.sub(r'\s+', ' ', paragraph)
        paragraph = doc['title'] + "\n" + paragraph
        model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
        embedding = model.encode(paragraph)
        document = {
            "source": "web",
            "url": doc.get("url", ''),
            "embedding": embedding.tolist(),
            "data": paragraph
        }
        embedding_collection.insert_one(document)
        print(f"Saved embedding data into MongoDB for URL: {doc.get('url', '')}")

In [14]:
featurize_web_data()

Saved embedding data into MongoDB for URL: https://docs.ros.org/en/galactic/Installation.html
Saved embedding data into MongoDB for URL: https://docs.ros.org/en/galactic/Installation/Ubuntu-Install-Debians.html
Saved embedding data into MongoDB for URL: https://docs.ros.org/en/galactic/Installation/Windows-Install-Binary.html
Saved embedding data into MongoDB for URL: https://docs.ros.org/en/galactic/Installation/RHEL-Install-RPMs.html
Saved embedding data into MongoDB for URL: https://docs.ros.org/en/galactic/Installation/Alternatives.html
Saved embedding data into MongoDB for URL: https://docs.ros.org/en/galactic/Installation/Alternatives/Ubuntu-Development-Setup.html
Saved embedding data into MongoDB for URL: https://docs.ros.org/en/galactic/Installation/Alternatives/Ubuntu-Install-Binary.html
Saved embedding data into MongoDB for URL: https://docs.ros.org/en/galactic/Installation/Alternatives/Windows-Development-Setup.html
Saved embedding data into MongoDB for URL: https://docs.ros

In [23]:
qdrant = QdrantClient(url="http://localhost:6333")

In [24]:
def insert_embeddings_to_qdrant(collection_name):
    mongo_client = MongoClient(MONGO_URI)
    db = mongo_client[DB_NAME]
    embedding_collection = db[EMBEDDINGS_COLLECTION_NAME]

    
    my_collection = collection_name
    qdrant.create_collection(
    collection_name=my_collection,
    vectors_config=models.VectorParams(size=384, distance=models.Distance.COSINE)
    )
    web_data = embedding_collection.find({})
    for i, doc in enumerate(web_data):

        embedding = doc['embedding']
        source = doc['source']
        url = doc['url']
        data = doc['data']
        qdrant.upsert(
            collection_name= my_collection,
            points=[models.PointStruct(
                id=i,
                vector=embedding,
                payload={"data": data, "source": source, "url": url}
            )]
        )
    print("Inserted all embeddings into Qdrant")


        
    
    

In [25]:
insert_embeddings_to_qdrant("text_collection")

ResponseHandlingException: [WinError 10061] No connection could be made because the target machine actively refused it

In [22]:
# qdrant = QdrantClient(":memory:")
qdrant.count(
    collection_name="text_collection",
    exact=True,
)

CountResult(count=2583)